<a href="https://colab.research.google.com/github/SolarHO/FloodDamage_BigData/blob/main/%EC%B9%A8%EC%88%98%ED%9D%94%EC%A0%81%EB%8F%84/%EC%B9%A8%EC%88%98%ED%9D%94%EC%A0%81%EB%8F%84_%EC%8B%9C%EA%B0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 기본 지도 생성(m)

In [ ]:
import folium
import pandas as pd
import geopandas as gpd
import json

In [ ]:
# 기본 지도 생성 (중앙 좌표와 줌 레벨은 임의로 설정)
m = folium.Map(location=[37.5665, 126.978], zoom_start=11)  # 서울 중심으로 설정

# 등고선을 직관적으로 보기 위해서 terrain 지도 사용
tiles = "https://tiles.stadiamaps.com/tiles/stamen_terrain/{z}/{x}/{y}{r}.png?api_key=c2e2edbf-1076-4f47-bfbc-e2df240577af"
attr = "terrain"

m = folium.Map(
    location=[37.5665, 126.978],
    zoom_start=11,
    tiles=tiles,
    attr=attr
)

## 2. 지도에 서울시 지역 레이어 표시

In [ ]:
s_geo='https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

# GeoJSON 데이터를 지도에 추가
folium.GeoJson(
    s_geo,
    style_function=lambda feature: {
        'color': 'red',  # 테두리 색
        'weight': 2,        # 테두리 두께
        'fillColor': 'red',  # 채우기 색
        'fillOpacity': 0.1   # 채우기 투명도
    }
).add_to(m)

## 4) 추출한 침수흔적도 시각화

In [ ]:
# GeoDataFrame으로 shp 파일 읽기
gdf = gpd.read_file("FloodMap.shp")

#geojson 형식으로 변환(좌표계를 4326)
gdf = gdf.to_crs(epsg=4326)
geojson_data = gdf.to_json()

# geojson_data를 파일로 저장
with open("FloodMap.geojson", "w", encoding="utf-8") as f:
    json.dump(json.loads(geojson_data), f, ensure_ascii=False, indent=4)

In [ ]:
# 침수흔적도 geojson 파일을 읽어옴
with open("FloodMap.geojson", 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

In [ ]:
# 침수 범례에 따라 색상으로 구분
def get_color(FLUD_GD):
    color_map = {
        1: (255, 255, 127),   #1등급(0.5m미만) 노란색
        2: (223, 255, 127),   #2등급(0.5m~1.0m미만) 녹황색
        3: (127, 255, 192),   #3등급(1.0m~1.5m미만) 연녹색
        4: (127, 200, 200),   #4등급(1.5m~2.0m미만) 청녹색
        5: (127, 200, 255),   #5등급(2.0m~3.0m미만) 연청색
        6: (127, 127, 255)    #6등급(3.0m이상) 고동색
    }
    return color_map.get(FLUD_GD, (169, 169, 169))  # 기본값 'gray' (169, 169, 169)

def style_function(feature):
    FLUD_GD = feature['properties']['FLUD_GD']
    rgb_color = get_color(FLUD_GD)
    return {
        'fillColor': f'rgba{rgb_color + (0.9,)}',
        'weight': 1,
        'opacity': 1,
        'fillOpacity': 0.6
    }

# GeoJSON 데이터를 지도에 추가
folium.GeoJson(
    geojson_data,
    style_function=style_function
).add_to(m)

# 지도 출력
m